##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras による分散型トレーニング

<table class="tfo-notebook-buttons" align="left">
  <td> <img src="https://www.tensorflow.org/images/tf_logo_32px.png"><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/keras.ipynb">TensorFlow.org で表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/colab_logo_32px.png"><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/distribute/keras.ipynb">Google Colab で実行</a> </td>
  <td> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/distribute/keras.ipynb">GitHub でソースを表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/download_logo_32px.png"><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/distribute/keras.ipynb">ノートブックをダウンロード</a> </td>
</table>

## 概要

`tf.distribute.Strategy` API は、複数の処理ユニットに渡ってトレーニングを分散するための抽象化を提供します。ユーザーは既存のモデルとトレーニングコードを使用して、最小限の変更で分散型トレーニングを実行できるようになります。

このチュートリアルでは、`tf.distribute.MirroredStrategy` を使用して、*1 台のマシンの多数の GPU で同期トレーニング*を行うグラフ内レプリケーションを実行します。ストラテジーは基本的にモデルのすべての変数を各プロセッサにコピーします。その後、[all-reduce](http://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/) を使用して全プロセッサからの勾配を結合し、結合された値をモデルの全コピーに適用します。

`tf.keras` API を使用して、モデルとそれをトレーニングするための `Model.fit` をビルドします。（カスタムトレーニングループと `MirroredStrategy` を使った分散型トレーニングについては、[こちらのチュートリアル](custom_training.ipynb)をご覧ください。）

`MirroredStrategy` は単一のマシンの複数の GPU でモデルをトレーニングします。*複数のワーカーの多数の GPU で同期トレーニング*を行う場合は、`tf.distribute.MultiWorkerMirroredStrategy` と[Keras の Model.fit](multi_worker_with_keras.ipynb) か[カスタムトレーニングループ](multi_worker_with_ctl.ipynb)を使用します。その他のオプションについては、[分散型トレーニングガイド](../../guide/distributed_training.ipynb)をご覧ください。

その他のさまざまなストラテジーについては、[TensorFlow の分散型トレーニング](../../guide/distributed_training.ipynb)ガイドをご覧ください。

## セットアップ

In [ ]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

In [ ]:
print(tf.__version__)

## データセットをダウンロードする

[TensorFlow Datasets](https://www.tensorflow.org/datasets) から MNIST データセットを読み込みます。これは、`tf.data` 形式のデータセットを返します。

`with_info` 引数を `True` に設定すると、データセット全体に対するメタデータが含まれます。ここでは `info` に保存されます。このメタデータオブジェクトには、トレーニングとテストの例の数などが含まれます。

In [ ]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

## 分散ストラテジーを定義する

`MirroredStrategy` オブジェクトを作成します。これは分散を処理し、モデル内に構築するコンテキストマネージャ (`MirroredStrategy.scope`) を提供します。

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

## Set up the input pipeline

When training a model with multiple GPUs, you can use the extra computing power effectively by increasing the batch size. In general, use the largest batch size that fits the GPU memory and tune the learning rate accordingly.

In [ ]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

画像ピクセル値を `[0, 255]` の範囲から `[0, 1]` の範囲に正規化する関数を定義します（[特徴量スケーリング](https://en.wikipedia.org/wiki/Feature_scaling)）。

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

この `scale` 関数をトレーニングとテストのデータに適用してから、`tf.data.Dataset` API を使用してトレーニングデータをシャッフル（`Dataset.shuffle`）し、バッチ化（`Dataset.batch`）します。パフォーマンスを改善するために、トレーニングデータのインメモリキャッシュも保持していることに注意してください（`Dataset.cache`）。

In [ ]:
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

## モデルを作成する

`Strategy.scope` のコンテキスト内で、Keras API を使ってモデルを作成し、コンパイルします。

In [ ]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

## コールバックを定義する


以下の [Keras コールバック](https://www.tensorflow.org/guide/keras/train_and_evaluate)を定義します。

- `tf.keras.callbacks.TensorBoard`: グラフを視覚化できるように、TensorBoard 用のログを書き込みます。
- `tf.keras.callbacks.ModelCheckpoint`: 各エポック後など、特定の頻度でモデルを保存します。
- `tf.keras.callbacks.BackupAndRestore`: モデルと現在のエポック番号をバックアップすることで、フォールトトレランス機能を提供します。詳細は、[Keras によるマルチワーカートレーニング](multi_worker_with_keras.ipynb)チュートリアルの*フォールトトレランス*セクションをご覧ください。
- `tf.keras.callbacks.LearningRateScheduler`: schedules the learning rate to change after, for example, every epoch/batch.

このノートブックでは例示目的で、`PrintLR` という[カスタムコールバック](https://www.tensorflow.org/guide/keras/custom_callback)を追加して、*学習率*を表示します。

**注意:** ジョブの失敗から再開する際に、トレーニング状態をリストアするための主なメカニズムとして、`ModelCheckpoint` の代わりに `BackupAndRestore` コールバックを使用してください。`BackupAndRestore` は eager モードのみをサポートするため、graph モードでは `ModelCheckpoint` を使用することを検討してください。

In [ ]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [ ]:
# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch &lt; 3:
    return 1e-3
  elif epoch &gt;= 3 and epoch &lt; 7:
    return 1e-4
  else:
    return 1e-5

In [ ]:
# Define a callback for printing the learning rate at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print('\nLearning rate for epoch {} is {}'.format(        epoch + 1, model.optimizer.lr.numpy()))

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

## トレーニングして評価する

次に、通常の方法でモデルをトレーニングします。モデル上で Keras `Model.fit` を呼び出し、チュートリアルの最初に作成したデータセットを渡します。トレーニングを分散しているかに関わらず、このステップは同じです。

In [ ]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

保存済みのチェックポイントを確認します。

In [ ]:
# check the checkpoint directory
!ls {checkpoint_dir}

モデルがどれほどうまく実行するかを確認するために、最新のチェックポイントを読み込み、テストデータで `Model.evaluate` を呼び出します。

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

eval_loss, eval_acc = model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

出力を視覚化するために、TensorBoard を起動して、ログを表示します。

In [ ]:
%tensorboard --logdir=logs

<!-- <img class="tfo-display-only-on-site" src="images/tensorboard_distributed_training_with_keras.png"/> -->

In [ ]:
!ls -sh ./logs

## SavedModel にエクスポートする

Keras の `Model.save` を使って、グラフと変数をプラットフォームに依存しない SavedModel 形式にエクスポートします。モデルが保存されたら、`Strategy.scope` の有無に関係なくそれを読み込めるようになります。

In [ ]:
path = 'saved_model/'

In [ ]:
model.save(path, save_format='tf')

次に、`Strategy.scope` を使用せずにモデルを読み込みます。

In [ ]:
unreplicated_model = tf.keras.models.load_model(path)

unreplicated_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

`Strategy.scope` を使用してモデルを読み込みます。

In [ ]:
with strategy.scope():
  replicated_model = tf.keras.models.load_model(path)
  replicated_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           optimizer=tf.keras.optimizers.Adam(),
                           metrics=['accuracy'])

  eval_loss, eval_acc = replicated_model.evaluate(eval_dataset)
  print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

### Additional resources

さまざまな分散ストラテジーと Keras `Model.fit` API を使用したその他の例をご覧ください。

1. [TPU で BERT を使って GLUE タスクを解決する](https://www.tensorflow.org/text/tutorials/bert_glue)チュートリアルでは、GPU でのトレーニングには `tf.distribute.MirroredStrategy` を使用し、TPU では `tf.distribute.TPUStrategy` を使用しています。
2. [分散ストラテジーを使ってモデルを保存して読み込む](save_and_load.ipynb)チュートリアルでは、SavedModel API と `tf.distribute.Strategy` の使用方法が説明されています。
3. [TensorFlow 公式モデル](https://github.com/tensorflow/models/tree/master/official)は、複数の分散ストラテジーを実行できるように構成可能です。

TensorFlow 分散ストラテジーに関してさらに学習するには、以下をご覧ください。

1. [tf.distribute.Strategy によるカスタムトレーニング](custom_training.ipynb)チュートリアルでは、カスタムトレーニングループを使って単一ワーカートレーニングに `tf.distribute.MirroredStrategy` を使用する方法が説明されています。
2. [Keras によるマルチワーカートレーニング](multi_worker_with_keras.ipynb)のチュートリアルでは、`MultiWorkerMirroredStrategy` と `Model.fit` を使用する方法が説明されています。
3. [Keras によるカスタムトレーニングループと MultiWorkerMirroredStrategy](multi_worker_with_ctl.ipynb) のチュートリアルでは、Keras とカスタムトレーニングループで`MultiWorkerMirroredStrategy` を使用する方法が説明されています。
4. [TensorFlow での分散型トレーニング](https://www.tensorflow.org/guide/distributed_training)ガイドでは、利用可能な分散ストラテジーの概要が説明されています。
5. [tf.function を使ったパフォーマンスの改善](../../guide/function.ipynb)ガイドでは、その他のストラテジーや、TensorFlow モデルのパフォーマンスを最適化するために使用できる [TensorFlow Profiler](../../guide/profiler.md) といったツールに関する情報が提供されています。

注意: `tf.distribute.Strategy` の開発は積極積に進められています。近日中にはより多くの例やチュートリアルを追加する予定ですので、ぜひお試しください。フィードバックをお待ちしております。[GitHub の課題](https://github.com/tensorflow/tensorflow/issues/new)から、お気軽にお寄せください。